# Feature Analysis for Emotion Classification

This notebook analyzes OpenSmile eGeMAPSv02 features and their relationship to emotions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append('..')

from src.config import PROCESSED_DATA_DIR, EMOTION_LABELS

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Load Processed Features

In [ ]:
# Load training data
X_train = np.load(PROCESSED_DATA_DIR / 'X_train.npy')
y_train = np.load(PROCESSED_DATA_DIR / 'y_train.npy')

print(f"Features shape: {X_train.shape}")
print(f"Labels shape: {y_train.shape}")
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of samples: {X_train.shape[0]}")

## 2. Feature Statistics

In [ ]:
# Basic statistics
df_features = pd.DataFrame(X_train)
print("Feature Statistics:")
print(df_features.describe())

# Check for NaN or inf values
print(f"\nNaN values: {np.isnan(X_train).sum()}")
print(f"Inf values: {np.isinf(X_train).sum()}")

In [ ]:
# Feature distribution
fig, axes = plt.subplots(4, 4, figsize=(16, 12))
axes = axes.flatten()

for i in range(min(16, X_train.shape[1])):
    axes[i].hist(X_train[:, i], bins=30, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'Feature {i}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 3. Feature Correlation Analysis

In [ ]:
# Compute correlation matrix
# Sample subset for visualization (all 88 features is too large)
sample_features = X_train[:, :20]
corr_matrix = np.corrcoef(sample_features.T)

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix (First 20 Features)', fontsize=14)
plt.tight_layout()
plt.show()

# Identify highly correlated features
high_corr = np.where(np.abs(corr_matrix) > 0.9)
high_corr_pairs = [(i, j) for i, j in zip(*high_corr) if i < j]
print(f"\nHighly correlated feature pairs (>0.9): {len(high_corr_pairs)}")

## 4. Dimensionality Reduction - PCA

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Explained variance
explained_var = pca.explained_variance_ratio_
cumulative_var = np.cumsum(explained_var)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].bar(range(1, len(explained_var[:20])+1), explained_var[:20], alpha=0.7)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('PCA Scree Plot (First 20 Components)')
axes[0].grid(alpha=0.3)

# Cumulative variance
axes[1].plot(range(1, len(cumulative_var)+1), cumulative_var, marker='o', markersize=3)
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Explained Variance')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Find components for 95% variance
n_components_95 = np.argmax(cumulative_var >= 0.95) + 1
print(f"\nComponents needed for 95% variance: {n_components_95}/{X_train.shape[1]}")
print(f"Dimension reduction: {(1 - n_components_95/X_train.shape[1])*100:.1f}%")

## 5. Feature Space Visualization

In [ ]:
# PCA 2D projection
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

# Create labels for visualization (use primary emotion)
primary_emotions = [EMOTION_LABELS[i] for i in np.argmax(y_train, axis=1)]

plt.figure(figsize=(12, 8))
for emotion in set(primary_emotions):
    mask = np.array(primary_emotions) == emotion
    plt.scatter(X_pca_2d[mask, 0], X_pca_2d[mask, 1], 
                label=emotion, alpha=0.6, s=50)

plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% variance)')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% variance)')
plt.title('Feature Space Visualization (PCA)', fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 6. t-SNE Visualization (if time permits)

In [ ]:
# Use subset for faster computation
n_samples = min(500, len(X_scaled))
X_subset = X_scaled[:n_samples]
y_subset = primary_emotions[:n_samples]

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_subset)

plt.figure(figsize=(12, 8))
for emotion in set(y_subset):
    mask = np.array(y_subset) == emotion
    plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
                label=emotion, alpha=0.6, s=50)

plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.title('Feature Space Visualization (t-SNE)', fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Feature Importance via Variance

In [ ]:
# Features with highest variance (after scaling)
feature_variance = np.var(X_scaled, axis=0)
top_features_idx = np.argsort(feature_variance)[-20:][::-1]

plt.figure(figsize=(12, 6))
plt.bar(range(len(top_features_idx)), feature_variance[top_features_idx])
plt.xlabel('Feature Index')
plt.ylabel('Variance')
plt.title('Top 20 Features by Variance')
plt.xticks(range(len(top_features_idx)), top_features_idx, rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("Top 10 features by variance:")
for i, idx in enumerate(top_features_idx[:10]):
    print(f"  {i+1}. Feature {idx}: variance={feature_variance[idx]:.4f}")

## 8. Conclusions

From the feature analysis:

1. **Feature Quality**: No NaN/Inf values, distributions look reasonable ✅
2. **Dimensionality**: Can reduce to ~40-50 components for 95% variance
3. **Separability**: Some emotion clusters visible in PCA/t-SNE
4. **Correlations**: Some features are highly correlated (expected for eGeMAPSv02)
5. **Recommendation**: StandardScaler normalization is essential for training